In [4]:
import pandas as pd
tsv_file = "spoc-train-train.tsv"
csv_file = "Train_1.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)
tsv_file = "spoc-testp.tsv"
csv_file = "Train_2.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)
tsv_file = "spoc-testw.tsv"
csv_file = "Train_3.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)
tsv_file = "spoc-train-eval.tsv"
csv_file = "Train_4.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)
tsv_file = "spoc-train-test.tsv"
csv_file = "Train_5.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)
tsv_file = "spoc-train.tsv"
csv_file = "Train_6.csv"
df = pd.read_csv(tsv_file, sep='\t')
df.to_csv(csv_file, index=False)

In [6]:
import pandas as pd
train_1 = 'Train_1.csv' 
train_2 = 'Train_2.csv'
train_3 = 'Train_3.csv'
train_4 = 'Train_4.csv'
train_5 = 'Train_5.csv'
train_6 = 'Train_6.csv'
train_data_1 = pd.read_csv(train_1)
train_data_2 = pd.read_csv(train_2)
train_data_3 = pd.read_csv(train_3)
train_data_4 = pd.read_csv(train_4)
train_data_5 = pd.read_csv(train_5)
train_data_6 = pd.read_csv(train_6)
merged_data = pd.concat([train_data_1, train_data_2, train_data_3,train_data_4,train_data_5,train_data_6], ignore_index=True)
merged_file = 'merged_data.csv'
merged_data.to_csv(merged_file, index=False)
print(f"Data has been successfully merged and saved to {merged_file}.")

Data has been successfully merged and saved to merged_data.csv.


In [7]:
df = pd.read_csv("merged_data.csv")

### Removing Column

In [8]:
df = df.drop(columns=['workerid'])
df = df.drop(columns=['probid'])
df = df.drop(columns=['line'])
df = df.drop(columns=['indent'])
df = df.drop(columns=['subid'])

## Text Tokenization

In [9]:
import pandas as pd
def tokenize_text(text):
    if pd.isna(text):
        return []
    text = text.replace(',', ' ')
    return text.split()
def tokenize_code(text):
    if pd.isna(text):
        return []
    text = text.replace(',', ' ')
    return text.split()

df['tokens'] = df['text'].apply(tokenize_text)
df['code_token'] = df['code'].apply(tokenize_code)
df = df.drop(columns=['text'])
df = df.drop(columns=['code'])


In [10]:
df.to_csv("tokenized_dataset.csv", index=False)

### Adding Necessary Token

In [11]:
def add_start_end_tokens(tokens):
    return ['<start>'] + tokens + ['<end>']
df['code_token'] = df['code_token'].apply(add_start_end_tokens)
max_length = max(df['tokens'].apply(len).max(), df['code_token'].apply(len).max())
def pad_tokens(tokens, max_length):
    return tokens + ['<pad>'] * (max_length - len(tokens))
df['tokens'] = df['tokens'].apply(lambda x: pad_tokens(x, max_length))
df['code_token'] = df['code_token'].apply(lambda x: pad_tokens(x, max_length))

In [13]:
df.to_csv("tokenized_dataset.csv", index=False)

## Vocablury

In [14]:
import pandas as pd
import json
import ast
data_path = "tokenized_dataset.csv"
df = pd.read_csv(data_path, header=None, names=["tokens", "code_token"])
vocab = set()
for col in ["tokens", "code_token"]:
    for text in df[col]:
        try:
            words = ast.literal_eval(text)  # Safely convert string to list
            vocab.update(words)
        except (SyntaxError, ValueError):
            print(f"Skipping malformed entry: {text}")
special_tokens = {"<pad>": 0, "<start>": 1, "<end>": 2, "<UNK>": 3}
vocab.discard("<pad>")
vocab.discard("<start>")
vocab.discard("<end>")
vocab.discard("<UNK>")
word_to_index = {word: idx for idx, word in enumerate(sorted(vocab), start=4)}
word_to_index = {**special_tokens, **word_to_index}
json_path = "vocabulary.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(word_to_index, f, ensure_ascii=False, indent=4)
print(f"Vocabulary saved to {json_path}")

Skipping malformed entry: tokens
Skipping malformed entry: code_token
Vocabulary saved to vocabulary.json


In [15]:
import json
with open('vocabulary.json', 'r', encoding='utf-8') as f:
    vocab_dict = json.load(f)
word = "create"
word_index = vocab_dict.get(word, None)
print(f"Index of '{word}': {word_index}")
reverse_vocab_dict = {v: k for k, v in vocab_dict.items()}
indices = [3, 4, 5, 6]  # Example word indices
decoded_words = [reverse_vocab_dict.get(idx, "<UNK>") for idx in indices]
print(f"Decoded words: {' '.join(decoded_words)}")


Index of 'create': 29038
Decoded words: <UNK> ! !!((N !!(K


### Creating Sequence

In [16]:
import csv
import json
vocab_path = 'vocabulary.json'
tokenized_data_path = 'tokenized_dataset.csv'
output_csv_path = 'sequences.csv'
with open(vocab_path, 'r', encoding='utf-8') as f:
    vocabulary = json.load(f)
def text_to_sequence(text, vocab):
    sequence = []
    for word in text:
        word_cleaned = word.strip()
        if word_cleaned in vocab:
            sequence.append(vocab[word_cleaned])
        else:
            print(f"Unmatched token: '{word_cleaned}'")
            sequence.append(-1)
    return sequence
with open(tokenized_data_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    output_data = []
    for row in reader:
        input_text = eval(row['tokens'])
        output_text = eval(row['code_token'])
        input_sequence = text_to_sequence(input_text, vocabulary)
        output_sequence = text_to_sequence(output_text, vocabulary)
        output_data.append({
            'tokens': input_sequence,
            'code_token': output_sequence
        })
with open(output_csv_path, 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['tokens', 'code_token']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in output_data:
        writer.writerow({
            'tokens': row['tokens'],
            'code_token': row['code_token']
        })
print(f"Converted sequences have been saved to {output_csv_path}.")


Converted sequences have been saved to sequences.csv.


### Data loader

In [17]:
from torch.utils.data import DataLoader, TensorDataset,Dataset
import pandas as pd
import torch
import ast
from torch.nn.utils.rnn import pad_sequence
class DataLoad(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.inputs = [ast.literal_eval(x) for x in df['tokens']]
    self.outputs = [ast.literal_eval(x) for x in df['code_token']]
  def __len__(self):
    return len(self.inputs)
  def __getitem__(self,idx):
    input_tensor = torch.tensor(self.inputs[idx], dtype=torch.int64)
    output_tensor = torch.tensor(self.outputs[idx],dtype=torch.int64)
    return input_tensor, output_tensor
def Add_Pad(batch):
  inputs,outputs = zip(*batch)
  inputs = pad_sequence(inputs,batch_first=True,padding_value=0)
  outputs = pad_sequence(outputs,batch_first=True,padding_value=0)
  return inputs,outputs
batch_size=32
dataset = DataLoad('sequences.csv')
dataloader = DataLoader(dataset,batch_size, shuffle=True,collate_fn=Add_Pad)
data_iter = iter(dataloader)
features, labels = next(data_iter)

In [18]:
import json
with open('vocabulary.json', 'r', encoding='utf-8') as f:
    vocab_dict = json.load(f)
Vocab_size = len(vocab_dict)


In [19]:
print("Features:\n", features)
print("Labels:\n", labels)

Features:
 tensor([[52131, 45942,     0,  ...,     0,     0,     0],
        [30196, 45942, 25334,  ...,     0,     0,     0],
        [52131, 45942,     0,  ...,     0,     0,     0],
        ...,
        [36884, 54530, 37953,  ...,     0,     0,     0],
        [18997, 14193, 37805,  ...,     0,     0,     0],
        [31005, 44428,  1956,  ...,     0,     0,     0]])
Labels:
 tensor([[    1, 27465, 14288,  ...,     0,     0,     0],
        [    1, 45942, 11646,  ...,     0,     0,     0],
        [    1, 27465, 14288,  ...,     0,     0,     0],
        ...,
        [    1, 36884,  9326,  ...,     0,     0,     0],
        [    1, 37546, 18998,  ...,     0,     0,     0],
        [    1, 28958, 14169,  ...,     0,     0,     0]])


### Transformer

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=60):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # Shape: (1, max_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class TransformerModel(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, d_model=256, nhead=8, num_layers=4, dim_feedforward=512, max_len=60):
        super(TransformerModel, self).__init__()

        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model, padding_idx=0)
        self.decoder_embedding = nn.Embedding(output_vocab_size, d_model, padding_idx=0)
        self.positional_encoding = PositionalEncoding(d_model, max_len)

        self.transformer = nn.Transformer(
            d_model=d_model, 
            nhead=nhead, 
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers, 
            dim_feedforward=dim_feedforward,
            dropout=0.1
        )

        self.fc_out = nn.Linear(d_model, output_vocab_size)

    def generate_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
        return mask.to(device)

    def forward(self, src, tgt):
        src_mask = None
        tgt_mask = self.generate_mask(tgt.size(1))

        src_key_padding_mask = src == 0
        tgt_key_padding_mask = tgt == 0

        src_emb = self.positional_encoding(self.encoder_embedding(src))
        tgt_emb = self.positional_encoding(self.decoder_embedding(tgt))

        output = self.transformer(
            src_emb.permute(1, 0, 2), 
            tgt_emb.permute(1, 0, 2),
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

        output = self.fc_out(output.permute(1, 0, 2))
        return output
input_vocab_size = Vocab_size  
output_vocab_size = Vocab_size  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_vocab_size, output_vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=5e-4)


C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [22]:
with open("vocabulary.json", "r") as f:
    vocab = json.load(f)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 513398: character maps to <undefined>

## Make Prediction

In [24]:
import json
import torch
from tqdm import tqdm



with open("vocabulary.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)


# Create token-to-index and index-to-token mappings
token2idx = vocab
idx2token = {idx: token for token, idx in vocab.items()}

def tokenize(sentence, token2idx):
    """Tokenizes the input sentence using the vocabulary."""
    return [token2idx.get(token, token2idx["<UNK>"]) for token in sentence.split()]

def detokenize(indices, idx2token):
    """Converts token indices back to words."""
    return " ".join([idx2token.get(idx, "<UNK>") for idx in indices])

def predict(model, sentence, max_len=60):
    """Generates C++ code from pseudocode or vice versa."""
    model.eval()
    
    # Tokenize input and convert to tensor
    input_tokens = tokenize(sentence, token2idx)
    input_tensor = torch.tensor(input_tokens).unsqueeze(0).to(device)  # Shape: (1, seq_len)

    # Start with the <sos> token for decoding
    output_tokens = [token2idx["<start>"]]
    
    with torch.no_grad():
        for _ in tqdm(range(max_len), desc="Decoding", leave=True):
            output_tensor = torch.tensor(output_tokens).unsqueeze(0).to(device)  # Shape: (1, seq_len)

            # Generate next token
            predictions = model(input_tensor, output_tensor)  # Shape: (1, seq_len, vocab_size)
            next_token = predictions[0, -1].argmax(dim=-1).item()

            # Stop if <eos> is generated
            if next_token == token2idx["<end>"]:
                break

            # Append next token to output sequence
            output_tokens.append(next_token)

    # Convert token indices back to words
    return detokenize(output_tokens[1:], idx2token)  # Exclude <sos> token




## Traning Loop

In [25]:
import time
import torch
from tqdm import tqdm

# Define training parameters
num_epochs = 10
clip_grad = 1.0  # Gradient clipping
batch_size = 10  # Adjust as needed

# Move model to device
model.to(device)

# Training function
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()  # Set to training mode

    for epoch in range(num_epochs):
        start_time = time.time()
        epoch_loss = 0

        # Wrap dataloader with tqdm for progress visualization
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch+1}")

        for batch_idx, (features, labels) in progress_bar:
            features, labels = features.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            output = model(features, labels[:, :-1])  # Remove last token from labels
            output = output.reshape(-1, output_vocab_size)  # Reshape for loss calculation
            labels = labels[:, 1:].reshape(-1)  # Shift labels by one position

            # Compute loss
            loss = criterion(output, labels)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

            # Update weights
            optimizer.step()

            epoch_loss += loss.item()

            # Update tqdm progress bar with current loss
            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

        end_time = time.time()
        avg_loss = epoch_loss / len(dataloader)
        print(f"\nEpoch {epoch+1} completed in {end_time - start_time:.2f} sec, Avg Loss: {avg_loss:.4f}")

        # Run prediction after each epoch
        manual_input = "function to add two numbers"
        output = predict(model, manual_input)
        print(f"\nGenerated Output after Epoch {epoch+1}:\n{output}\n")
train_model(model, dataloader, criterion, optimizer, num_epochs=num_epochs)


Epoch 1:   9%|▉         | 1854/21084 [04:23<45:27,  7.05it/s, loss=2.2110]  


KeyboardInterrupt: 

### Save Model

In [19]:
import torch
model_save_path = "transformer_model.pt"
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict()
}, model_save_path)
print(f"✅ Model saved successfully at {model_save_path}")

✅ Model saved successfully at transformer_model.pt


## Load Model To Make Prediction

In [ ]:
import json
import torch
from tqdm import tqdm

# Load vocabulary
with open("vocabulary.json", "r", encoding="utf-8") as f:
    vocab = json.load(f)

# Create token-to-index and index-to-token mappings
token2idx = vocab
idx2token = {idx: token for token, idx in vocab.items()}

# Initialize model_2 (ensure it's the same architecture as the saved model)
model_2 = TransformerModel(input_vocab_size, output_vocab_size).to(device)  # Initialize with same parameters as trained model
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.001)  # Use same optimizer

# Function to load the model
def load_model(model_path, model_2, optimizer):
    checkpoint = torch.load(model_path, map_location=device)
    model_2.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    model_2.eval()  # Set to evaluation mode
    print(f"✅ Model loaded from {model_path}")

# Load the saved model
load_model("transformer_model.pt", model_2, optimizer)

# Tokenization functions
def tokenize(sentence, token2idx):
    """Tokenizes the input sentence using the vocabulary."""
    return [token2idx.get(token, token2idx["<UNK>"]) for token in sentence.split()]

def detokenize(indices, idx2token):
    """Converts token indices back to words."""
    return " ".join([idx2token.get(idx, "<UNK>") for idx in indices])

# Prediction function
def predict(model, sentence, max_len=60):
    """Generates C++ code from pseudocode or vice versa."""
    model.eval()
    
    # Tokenize input and convert to tensor
    input_tokens = tokenize(sentence, token2idx)
    input_tensor = torch.tensor(input_tokens).unsqueeze(0).to(device)  # Shape: (1, seq_len)

    # Start with the <sos> token for decoding
    output_tokens = [token2idx["<start>"]]
    
    with torch.no_grad():
        for _ in tqdm(range(max_len), desc="Decoding", leave=True):
            output_tensor = torch.tensor(output_tokens).unsqueeze(0).to(device)  # Shape: (1, seq_len)

            # Generate next token
            predictions = model(input_tensor, output_tensor)  # Shape: (1, seq_len, vocab_size)
            next_token = predictions[0, -1].argmax(dim=-1).item()

            # Stop if <eos> is generated
            if next_token == token2idx["<end>"]:
                break

            # Append next token to output sequence
            output_tokens.append(next_token)

    # Convert token indices back to words
    return detokenize(output_tokens[1:], idx2token)  # Exclude <sos> token

# Example input
manual_input = 'add My'
output = predict(model_2, manual_input)

print("\nGenerated Output:\n", output)


C:\Users\ahmed\AppData\Local\Temp\ipykernel_14532\1468499665.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


✅ Model loaded from transformer_model.pt


Decoding:   5%|▌         | 3/60 [00:00<00:00, 60.59it/s]


Generated Output:
 liter += my;


In [ ]:
# Example input
manual_input = "create string s"
output = predict(model, manual_input)

print("\nGenerated Output:\n", output)